In [1]:
import pandas as pd
import networkx as nx
from helpers import *

# Data Loading

In [2]:
# Articles is the name of all the articles in our dataset
# the index serves also as an id to retrieve articles in other files
# the names are url encoded
articles_df = pd.read_csv("./data/wikispeedia_paths-and-graph/articles.tsv",names = ["name"] ,skiprows=12,sep="\t")

#The categories of the articles, row index is the same as for the articles df
categories_df = pd.read_csv("./data/wikispeedia_paths-and-graph/categories.tsv",names = ["name","category"],skiprows=13, sep="\t")


#each row of of the links df contains the list of article names that an article links to
links_df = pd.read_csv("./data/wikispeedia_paths-and-graph/links.tsv",skiprows=12, names = ["from","to"],sep="\t")

#
shortest_path_distance = pd.read_csv("./data/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt",names=["distances"], skiprows=17)

#
paths_finished_df = pd.read_csv("./data/wikispeedia_paths-and-graph/paths_finished.tsv",names = ["hashedIpAddress","timestamp","durationInSec","path","rating"],skiprows=16, sep="\t")
#
paths_unfinished_df = pd.read_csv("./data/wikispeedia_paths-and-graph/paths_unfinished.tsv",names=["hashedIpAddress",   "timestamp ",  "durationInSec",  "path",   "target",   "type"],skiprows=17, sep="\t")


##### articles data frame

In [3]:
articles_df

,name
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in
1,%C3%85land
2,%C3%89douard_Manet
3,%C3%89ire
4,%C3%93engus_I_of_the_Picts
...,...
4599,Zionism
4600,Zirconium
4601,Zoroaster
4602,Zuid-Gelders


##### categories dataframe

In [4]:
categories_df

,name,category
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.History.British_History.British_Histor...
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,subject.People.Historical_figures
2,%C3%85land,subject.Countries
3,%C3%85land,subject.Geography.European_Geography.European_...
4,%C3%89douard_Manet,subject.People.Artists
...,...,...
5199,Zirconium,subject.Science.Chemistry.Chemical_elements
5200,Zoroaster,subject.People.Religious_figures_and_leaders
5201,Zuid-Gelders,subject.Geography.European_Geography
5202,Zuid-Gelders,subject.Language_and_literature.Languages


##### Links dataframe

In [5]:
links_df

,from,to
0,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Bede
1,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Columba
2,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,D%C3%A1l_Riata
3,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Great_Britain
4,%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,Ireland
...,...,...
119877,Zulu,South_Africa
119878,Zulu,Swaziland
119879,Zulu,United_Kingdom
119880,Zulu,Zambia


##### Shortest path distances

In [6]:
shortest_path_distance

,distances
0,0_____33333325634333435_2433544334_3_422343544...
1,_0____22222325623232424_2422544324_3_312242544...
2,__0___33222425623232324_2333444433_3_422343434...
3,___0__33333325634233334_2433434333_2_423343433...
4,____0_22323335633332435_2433545434_3_423343544...
...,...
4599,______22222325622231424_1322544334_3_422232544...
4600,______33333434523232434_2332544324_3_323333544...
4601,______22222424522231434_2322545434_3_422232544...
4602,______33333436733342435_2433545444_3_523353544...


##### Finished Paths dataframe

In [7]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1.0


##### Unfinished Paths dataframe

In [8]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout
...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,Cholera,restart
24871,232f992e57d43e8d,1389787697,6,Modern_history,Hollandic,restart
24872,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,The_Beatles,timeout
24873,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,Alan_Turing,timeout


# Data Cleaning

### Cleaning categories

In [9]:
categories_grouped_df = categories_df.groupby(["name"]).agg(lambda x : list(x))
categories_grouped_df

,category
name,
%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,[subject.History.British_History.British_Histo...
%C3%85land,"[subject.Countries, subject.Geography.European..."
%C3%89douard_Manet,[subject.People.Artists]
%C3%89ire,"[subject.Countries, subject.Geography.European..."
%C3%93engus_I_of_the_Picts,[subject.History.British_History.British_Histo...
...,...
Zionism,"[subject.People.Political_People, subject.Reli..."
Zirconium,[subject.Science.Chemistry.Chemical_elements]
Zoroaster,[subject.People.Religious_figures_and_leaders]


We can see that not all the articles have corresponding categories, as we can see below

In [10]:
categories_joined_df = articles_df.join(categories_grouped_df,on="name",how="left")
categories_joined_df [categories_joined_df.category.isna()]

,name,category
1210,Directdebit,NaN
1231,Donation,NaN
1600,Friend_Directdebit,NaN
3253,Pikachu,NaN
3849,Sponsorship_Directdebit,NaN
4545,Wowpurchase,NaN


We will be using BERTopic, pretrained on the categories that we already have, to fill out the missing categories of these articles

### Played games paths

In [11]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0
...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,Yagan;Ancient_Egypt;Civilization,NaN
51314,2ef7ac844cefda58,1300254138,165,Yagan;Folklore;Brothers_Grimm;<;19th_century;C...,3.0
51315,12863abb7887f890,1385095372,228,Yagan;Australia;England;France;United_States;T...,NaN
51316,19f8284371753362,1298792567,56,Yarralumla%2C_Australian_Capital_Territory;Aus...,1.0


In [12]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type
0,2426091a53125110,1297054935,1804,Obi-Wan_Kenobi,Microsoft,timeout
1,26141fd878806294,1297055651,1805,Julius_Caesar,Caracas,timeout
2,2b015fb8181c48f2,1297090819,1818,Malawi;Democracy;Alexander_the_Great,First_Crusade,timeout
3,53a53bc244e08a6a,1297094761,49,Paraguay,Mount_St._Helens,restart
4,53a53bc244e08a6a,1297099105,1808,Paraguay;Bolivia,Mount_St._Helens,timeout
...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,Franz_Kafka;Tuberculosis;World_Health_Organiza...,Cholera,restart
24871,232f992e57d43e8d,1389787697,6,Modern_history,Hollandic,restart
24872,2e09a7224600a7cd,1389798400,1900,Computer_programming;Linguistics;Culture;Popul...,The_Beatles,timeout
24873,60af9e2138051b96,1389799481,1903,Jamaica;United_Kingdom;World_War_II;Battle_of_...,Alan_Turing,timeout


We can see that the path is formatted in a way we can't use efficiently, we will contruct the path as an array

In [13]:
paths_finished_df["path"] = paths_finished_df["path"].apply(lambda x: x.split(";"))
paths_unfinished_df["path"] = paths_unfinished_df["path"].apply(lambda x: x.split(";"))

We will also add a column containg the name of the article for the start and target paths for each game

In [14]:
paths_finished_df["start"] = paths_finished_df["path"].apply(lambda x: x[0])
paths_finished_df["target"] = paths_finished_df["path"].apply(lambda x: x[-1])

paths_unfinished_df["start"] = paths_unfinished_df["path"].apply(lambda x: x[0])

We also know that backs are also recorded in the path, these decisions that the user made could be interesting would be analyzed seprately, 
but for analysis, we will only focus on the final path that a player used to try to find the target.
To this end, we implemented a method (remove_backs, found in the helpers module) that would remove the "false" movements the player made

In [15]:
paths_finished_df["path"] = paths_finished_df["path"].apply(remove_backs)
paths_unfinished_df["path"] = paths_unfinished_df["path"].apply(remove_backs)

In [16]:
paths_finished_df

,hashedIpAddress,timestamp,durationInSec,path,rating,start,target
0,6a3701d319fc3754,1297740409,166,"[14th_century, 15th_century, 16th_century, Pac...",NaN,14th_century,African_slave_trade
1,3824310e536af032,1344753412,88,"[14th_century, Europe, Africa, Atlantic_slave_...",3.0,14th_century,African_slave_trade
2,415612e93584d30e,1349298640,138,"[14th_century, Niger, Nigeria, British_Empire,...",NaN,14th_century,African_slave_trade
3,64dd5cd342e3780c,1265613925,37,"[14th_century, Renaissance, Ancient_Greece, Gr...",NaN,14th_century,Greece
4,015245d773376aab,1366730828,175,"[14th_century, Italy, Roman_Catholic_Church, H...",3.0,14th_century,John_F._Kennedy
...,...,...,...,...,...,...,...
51313,15a13a1d66ef5456,1349231015,66,"[Yagan, Ancient_Egypt, Civilization]",NaN,Yagan,Civilization
51314,2ef7ac844cefda58,1300254138,165,"[Yagan, Folklore, 19th_century, Charles_Dicken...",3.0,Yagan,Fiction
51315,12863abb7887f890,1385095372,228,"[Yagan, Australia, England, France, United_Sta...",NaN,Yagan,U.S._Open_%28tennis%29
51316,19f8284371753362,1298792567,56,"[Yarralumla%2C_Australian_Capital_Territory, A...",1.0,Yarralumla%2C_Australian_Capital_Territory,Abraham_Lincoln


In [17]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type,start
0,2426091a53125110,1297054935,1804,[Obi-Wan_Kenobi],Microsoft,timeout,Obi-Wan_Kenobi
1,26141fd878806294,1297055651,1805,[Julius_Caesar],Caracas,timeout,Julius_Caesar
2,2b015fb8181c48f2,1297090819,1818,"[Malawi, Democracy, Alexander_the_Great]",First_Crusade,timeout,Malawi
3,53a53bc244e08a6a,1297094761,49,[Paraguay],Mount_St._Helens,restart,Paraguay
4,53a53bc244e08a6a,1297099105,1808,"[Paraguay, Bolivia]",Mount_St._Helens,timeout,Paraguay
...,...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,"[Franz_Kafka, Tuberculosis, World_Health_Organ...",Cholera,restart,Franz_Kafka
24871,232f992e57d43e8d,1389787697,6,[Modern_history],Hollandic,restart,Modern_history
24872,2e09a7224600a7cd,1389798400,1900,"[Computer_programming, Linguistics, Culture]",The_Beatles,timeout,Computer_programming
24873,60af9e2138051b96,1389799481,1903,"[Jamaica, United_Kingdom, World_War_II, Battle...",Alan_Turing,timeout,Jamaica


We also noticed that for the unfinished paths, a number of them were just unplayed games where players did not play at all
This behaviour was seen when there was no movements made by the player (i.e. no visited paths) and there was a timeout.
We decided to remove these games as they would not add any value to our analysis
We also have to note that there might be other games where players have started playing the game (i.e. visited some pages)
but then just decided to quit without exiting the game, so we decided to keep them as to not make assumptions about wether they quit the game or really did timeout on the game

In [18]:
paths_unfinished_df = paths_unfinished_df.drop(paths_unfinished_df[(paths_unfinished_df["durationInSec"]>=1800) & (paths_unfinished_df["path"].apply(lambda x: len(x)==1))].index)

In [19]:
paths_unfinished_df

,hashedIpAddress,timestamp,durationInSec,path,target,type,start
2,2b015fb8181c48f2,1297090819,1818,"[Malawi, Democracy, Alexander_the_Great]",First_Crusade,timeout,Malawi
3,53a53bc244e08a6a,1297094761,49,[Paraguay],Mount_St._Helens,restart,Paraguay
4,53a53bc244e08a6a,1297099105,1808,"[Paraguay, Bolivia]",Mount_St._Helens,timeout,Paraguay
5,131600803df4895e,1297100557,2009,"[Agriculture, History_of_the_world, China, Yan...",Grand_Canal_of_China,timeout,Agriculture
6,486bb79910fe9dd2,1297101660,1932,"[Mind, Christianity, Europe, Renaissance, Anci...",Scouting,timeout,Mind
...,...,...,...,...,...,...,...
24870,109ed71f571d86e9,1389787605,180,"[Franz_Kafka, Tuberculosis, World_Health_Organ...",Cholera,restart,Franz_Kafka
24871,232f992e57d43e8d,1389787697,6,[Modern_history],Hollandic,restart,Modern_history
24872,2e09a7224600a7cd,1389798400,1900,"[Computer_programming, Linguistics, Culture]",The_Beatles,timeout,Computer_programming
24873,60af9e2138051b96,1389799481,1903,"[Jamaica, United_Kingdom, World_War_II, Battle...",Alan_Turing,timeout,Jamaica


# Data Construction

In [ ]:
G = nx.from_pandas_edgelist(links_df,source="from", target="to", create_using=nx.DiGraph())

In [ ]:
We perform a sanity check to verify that the shortest paths correspond to what we have in the dataset

## Exploratory Data Analysis

We will first construct a column containing all the outer edges and the inner edges of an article

In [20]:
#Out going edges for an article
out_links_df = links_df.groupby(["from"]).agg(lambda x : list(x))
#Adding the articles where there is no outgoing edges
#out_links_df = pd.merge(articles_df,out_links_df,left_on="name",right_on="from",how="left") #could be removed
#out_links_df["to"]= out_links_df["to"].apply(lambda d: d if isinstance(d, list) else []) #could be removed

display(out_links_df)


#Incoming edges for an article
in_links_df = links_df.groupby(["to"]).agg(lambda x : list(x))
#Adding the articles where there is no incoming edges
#in_links_df = pd.merge(articles_df,in_links_df,left_on="name",right_on="to",how="left") #could be removed
#in_links_df["from"] = in_links_df["from"].apply(lambda d: d if isinstance(d, list) else []) #could be removed
display(in_links_df)

,to
from,
%C3%81ed%C3%A1n_mac_Gabr%C3%A1in,"[Bede, Columba, D%C3%A1l_Riata, Great_Britain,..."
%C3%85land,"[20th_century, Baltic_Sea, Crimean_War, Curren..."
%C3%89douard_Manet,"[Absinthe, Beer, Claude_Monet, Diego_Vel%C3%A1..."
%C3%89ire,"[Canada, English_language, George_VI_of_the_Un..."
%C3%93engus_I_of_the_Picts,"[D%C3%A1l_Riata, Durham, England, Great_Britai..."
...,...
Zionism,"[18th_century, 19th_century, Adolf_Hitler, Alb..."
Zirconium,"[Aluminium, Arabic_language, Australia, Bicycl..."
Zoroaster,"[18th_century, 9th_century, Afghanistan, Age_o..."


,from
to,
10th_century,"[11th_century, 12th_century, 15th_century, 16t..."
11th_century,"[10th_century, 12th_century, 15th_century, 16t..."
12th_century,"[11th_century, 13th_century, 15th_century, 16t..."
13th_century,"[11th_century, 12th_century, 14th_century, 15t..."
14th_century,"[11th_century, 12th_century, 13th_century, 15t..."
...,...
Zionism,"[19th_century, Abraham_Goldfaden, Albert_Einst..."
Zirconium,"[Calcium, Cerium, Diamond_simulant, Gas_metal_..."
Zoroaster,"[Friedrich_Nietzsche, Greco-Buddhism, Iran, Ir..."


In [21]:
out_links_df["num_links"]=out_links_df["to"].apply(lambda x: len(x))
out_links_df.sort_values(by="num_links",ascending=False).iloc[:10]

,to,num_links
from,,
United_States,"[Abraham_Lincoln, Advertising, Agriculture, Am...",294
Driving_on_the_left_or_right,"[Afghanistan, Africa, Akrotiri_and_Dhekelia, A...",255
List_of_countries,"[Abkhazia, Afghanistan, Akrotiri_and_Dhekelia,...",244
List_of_circulating_currencies,"[Abkhazia, Afghanistan, Akrotiri_and_Dhekelia,...",236
List_of_sovereign_states,"[Abkhazia, Afghanistan, African_Union, Albania...",216
Africa,"[Abidjan, Abuja, Accra, Addis_Ababa, African_G...",212
List_of_countries_by_system_of_government,"[Afghanistan, Albania, Algeria, Andorra, Angol...",207
Lebanon,"[Afghanistan, Africa, Agriculture, Albania, Al...",192
Interpol,"[Afghanistan, Albania, Algeria, Andorra, Angol...",191


In [22]:
in_links_df["num_links"]=in_links_df["from"].apply(lambda x: len(x))
in_links_df.sort_values(by="num_links",ascending=False).iloc[:10]

,from,num_links
to,,
United_States,"[%E2%82%AC2_commemorative_coins, 15th_Marine_E...",1551
United_Kingdom,"[%C3%85land, %C3%89ire, %E2%82%AC2_commemorati...",972
France,"[%C3%89douard_Manet, %E2%82%AC2_commemorative_...",959
Europe,"[13th_century, 14th_century, 15th_century, 16t...",933
England,"[%C3%93engus_I_of_the_Picts, 11th_century, 12t...",751
World_War_II,"[%C3%85land, 15th_Marine_Expeditionary_Unit, 1...",751
Germany,"[%C3%89douard_Manet, %E2%82%AC2_commemorative_...",743
India,"[10th_century, 11th_century, 12th_century, 14t...",611
English_language,"[%C3%89ire, A_Christmas_Carol, A_Wrinkle_in_Ti...",598


# Research Questions